In [1]:
import pandas as pd
import numpy as np
PATH = '/Users/orendar/Downloads/dataverse_files/'

In [2]:
train_df = pd.read_csv(f'{PATH}train_preds.csv', index_col = 0)
train_df.head()

,d.gone.neg.l1,d.gone.neg.l2,d.neg.frac.l3,week,camp.length,base.poll,y2000,y2002,y2004,y2006,...,deminc,dem.contrib.l1,dem.contrib.l2,rep.contrib.l1,rep.contrib.l2,d.gone.neg,gbc_preds,rfc_preds,etc_preds,nn_preds
1,0,0,0.000000,-11,23,45.744681,0,1,0,0,...,1,38350.0,126536.00,132545.00,102559.36,0,0.235871,0.146,0.0,0.199028
2,0,0,0.000000,-10,23,45.744681,0,1,0,0,...,1,58035.0,38350.00,522327.00,132545.00,0,0.154327,0.084,0.0,0.283316
3,1,0,0.000000,-8,23,45.744681,0,1,0,0,...,1,167280.0,67068.67,177739.44,138533.00,1,0.921011,0.958,1.0,0.906001
4,1,1,0.000000,-7,23,45.744681,0,1,0,0,...,1,389947.0,167280.00,543465.00,177739.44,1,0.985290,0.998,1.0,0.974159
5,1,1,0.071429,-6,23,45.744681,0,1,0,0,...,1,202734.0,389947.00,2415314.50,543465.00,1,0.989649,1.000,1.0,0.994450


In [3]:
test_df = pd.read_csv(f'{PATH}test_preds.csv',  index_col = 0)
test_df.head()

,d.gone.neg.l1,d.gone.neg.l2,d.neg.frac.l3,week,camp.length,base.poll,y2000,y2002,y2004,y2006,...,deminc,dem.contrib.l1,dem.contrib.l2,rep.contrib.l1,rep.contrib.l2,d.gone.neg,gbc_preds,rfc_preds,etc_preds,nn_preds
1,1,1,1.000000,-7,13,44.736842,0,1,0,0,...,0,41083.00,61295.00,289158.00,344029.00,1,0.944583,0.882,0.942,0.900369
2,0,0,0.000000,-6,23,63.636364,0,0,0,1,...,1,699168.06,175919.18,34550.00,8641.00,0,0.020580,0.006,0.010,0.020276
3,1,1,0.176471,-3,23,62.765957,0,1,0,0,...,0,383312.88,420975.89,31510.13,390706.48,1,0.902891,0.920,0.962,0.946103
4,0,0,0.090909,-2,27,62.637363,0,0,0,1,...,1,57150.00,34750.00,219018.00,194750.00,0,0.029230,0.156,0.078,0.028789
5,0,0,0.000000,-6,15,66.455237,0,0,0,1,...,1,165910.00,175460.08,91269.03,66217.39,0,0.151891,0.246,0.180,0.121091


In [4]:
target = "d.gone.neg"

y_train = train_df[target]
x_train = train_df.drop(target, axis=1)
y_test = test_df[target]
x_test = test_df.drop(target, axis=1)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((818, 33), (818,), (274, 33), (274,))

In [5]:
from sklearn.ensemble import *
from sklearn.metrics import *
rs = 42

In [6]:
gbc = GradientBoostingClassifier(n_estimators=100, max_features='sqrt', random_state=rs)
rfc = RandomForestClassifier(n_estimators=500, random_state=rs, n_jobs=-1)
etc = ExtraTreesClassifier(n_estimators=500, random_state=rs, n_jobs=-1)

In [7]:
gbc.fit(x_train, y_train)
rfc.fit(x_train, y_train)
etc.fit(x_train, y_train);

In [8]:
gbc_preds = gbc.predict(x_test)
rfc_preds = rfc.predict(x_test)
etc_preds = etc.predict(x_test)
gbc_prob_preds = gbc.predict_proba(x_test)
rfc_prob_preds = rfc.predict_proba(x_test)
etc_prob_preds = etc.predict_proba(x_test)
gbc_pos = gbc_prob_preds.T[1]
rfc_pos = rfc_prob_preds.T[1]
etc_pos = etc_prob_preds.T[1]

In [9]:
def rmse(preds): return mean_squared_error(y_test, preds)**.5
def auroc(preds): return roc_auc_score(y_test, preds)
def accuracy(preds): return accuracy_score(y_test, preds)
def logloss(preds): return log_loss(y_test, preds)

In [10]:
rmse(gbc_pos), rmse(rfc_pos), rmse(etc_pos)

(0.3601286977826547, 0.34268785140948554, 0.3012700851952889)

In [11]:
accuracy(gbc_preds), accuracy(rfc_preds), accuracy(etc_preds)

(0.864963503649635, 0.864963503649635, 0.8576642335766423)

In [12]:
auroc(gbc_pos), auroc(rfc_pos), auroc(etc_pos)

(0.9356060606060606, 0.9272360703812317, 0.9376221896383187)

In [13]:
logloss(gbc_pos), logloss(rfc_pos), logloss(etc_pos)

(1.032127670694763, 0.7937493292502644, 0.29190121457875523)

In [14]:
df = pd.DataFrame(data=[[accuracy(gbc_preds), rmse(gbc_pos), auroc(gbc_pos), logloss(gbc_pos)],
                        [accuracy(rfc_preds), rmse(rfc_pos), auroc(rfc_pos), logloss(rfc_pos)],
                        [accuracy(etc_preds), rmse(etc_pos), auroc(etc_pos), logloss(etc_pos)]],
                  index=['GBM', 'RF', 'ET'],
                  columns=['Accuracy', 'RMSE', 'AUROC', 'LogLoss'])
df.head()

,Accuracy,RMSE,AUROC,LogLoss
GBM,0.864964,0.360129,0.935606,1.032128
RF,0.864964,0.342688,0.927236,0.793749
ET,0.857664,0.301270,0.937622,0.291901


In [15]:
df.to_latex()

'\\begin{tabular}{lrrrr}\n\\toprule\n{} &  Accuracy &      RMSE &     AUROC &   LogLoss \\\\\n\\midrule\nGBM &  0.864964 &  0.360129 &  0.935606 &  1.032128 \\\\\nRF  &  0.864964 &  0.342688 &  0.927236 &  0.793749 \\\\\nET  &  0.857664 &  0.301270 &  0.937622 &  0.291901 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [16]:
from keras.models import *
from keras.layers import *
from keras.utils import *
from keras.optimizers import *
from clr import *
from sklearn.preprocessing import *
np.random.seed(42)

/Users/orendar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
x_train_mat, y_train_mat = x_train.as_matrix(), y_train.as_matrix().reshape(-1, 1)
x_test_mat, y_test_mat = x_test.as_matrix(), y_test.as_matrix().reshape(-1, 1)

In [18]:
scaler = StandardScaler()
x_train_norm = scaler.fit_transform(x_train_mat)
x_test_norm = scaler.transform(x_test_mat)

In [43]:
model = Sequential()
model.add(Dropout(0.15, input_shape=(33,)))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile model
opt = optimizers.adam(lr=5e-6, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
#opt = optimizers.SGD(lr=5e-4, momentum=0.9, decay=0, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['mse', 'accuracy'])

#callbacks arsenal
def sched(idx, lr): return lr*0.9 if idx % 10 == 0 else lr
lrsched = LearningRateScheduler(sched, verbose=0)
reducelr = ReduceLROnPlateau(factor=0.8, patience=2)
stop = EarlyStopping(patience=50)
clr = CyclicLR(base_lr=1e-6, max_lr=1e-5, step_size=20, mode='triangular', scale_mode='cycle')

In [44]:
# Fit the model
model.fit(x_train_norm, y_train, validation_data=(x_test_norm, y_test), epochs=3000, batch_size=512,
          callbacks=[clr, stop], verbose=2)

# evaluate the model
scores = model.evaluate(x_test_norm, y_test)
model.metrics_names, scores

Train on 818 samples, validate on 274 samples
Epoch 1/3000
 - 0s - loss: 0.6926 - mean_squared_error: 0.2498 - acc: 0.5098 - val_loss: 0.6958 - val_mean_squared_error: 0.2513 - val_acc: 0.4489
Epoch 2/3000
 - 0s - loss: 0.7007 - mean_squared_error: 0.2538 - acc: 0.4609 - val_loss: 0.6948 - val_mean_squared_error: 0.2508 - val_acc: 0.4599
Epoch 3/3000
 - 0s - loss: 0.6975 - mean_squared_error: 0.2521 - acc: 0.4927 - val_loss: 0.6935 - val_mean_squared_error: 0.2502 - val_acc: 0.4818
Epoch 4/3000
 - 0s - loss: 0.6928 - mean_squared_error: 0.2498 - acc: 0.5061 - val_loss: 0.6918 - val_mean_squared_error: 0.2493 - val_acc: 0.4891
Epoch 5/3000
 - 0s - loss: 0.6941 - mean_squared_error: 0.2505 - acc: 0.4988 - val_loss: 0.6896 - val_mean_squared_error: 0.2483 - val_acc: 0.4964
Epoch 6/3000
 - 0s - loss: 0.6896 - mean_squared_error: 0.2482 - acc: 0.5269 - val_loss: 0.6872 - val_mean_squared_error: 0.2470 - val_acc: 0.5073
Epoch 7/3000
 - 0s - loss: 0.6843 - mean_squared_error: 0.2456 - acc: 0.

Epoch 57/3000
 - 0s - loss: 0.5700 - mean_squared_error: 0.1900 - acc: 0.8325 - val_loss: 0.5801 - val_mean_squared_error: 0.1948 - val_acc: 0.8504
Epoch 58/3000
 - 0s - loss: 0.5667 - mean_squared_error: 0.1883 - acc: 0.8509 - val_loss: 0.5791 - val_mean_squared_error: 0.1943 - val_acc: 0.8540
Epoch 59/3000
 - 0s - loss: 0.5645 - mean_squared_error: 0.1876 - acc: 0.8337 - val_loss: 0.5784 - val_mean_squared_error: 0.1940 - val_acc: 0.8540
Epoch 60/3000
 - 0s - loss: 0.5687 - mean_squared_error: 0.1894 - acc: 0.8545 - val_loss: 0.5779 - val_mean_squared_error: 0.1937 - val_acc: 0.8540
Epoch 61/3000
 - 0s - loss: 0.5640 - mean_squared_error: 0.1871 - acc: 0.8716 - val_loss: 0.5776 - val_mean_squared_error: 0.1936 - val_acc: 0.8540
Epoch 62/3000
 - 0s - loss: 0.5685 - mean_squared_error: 0.1893 - acc: 0.8533 - val_loss: 0.5770 - val_mean_squared_error: 0.1933 - val_acc: 0.8540
Epoch 63/3000
 - 0s - loss: 0.5589 - mean_squared_error: 0.1847 - acc: 0.8643 - val_loss: 0.5762 - val_mean_squa

Epoch 113/3000
 - 0s - loss: 0.4867 - mean_squared_error: 0.1518 - acc: 0.8765 - val_loss: 0.5085 - val_mean_squared_error: 0.1623 - val_acc: 0.8650
Epoch 114/3000
 - 0s - loss: 0.4857 - mean_squared_error: 0.1517 - acc: 0.8765 - val_loss: 0.5071 - val_mean_squared_error: 0.1617 - val_acc: 0.8650
Epoch 115/3000
 - 0s - loss: 0.4818 - mean_squared_error: 0.1500 - acc: 0.8814 - val_loss: 0.5059 - val_mean_squared_error: 0.1612 - val_acc: 0.8650
Epoch 116/3000
 - 0s - loss: 0.4821 - mean_squared_error: 0.1502 - acc: 0.8778 - val_loss: 0.5050 - val_mean_squared_error: 0.1608 - val_acc: 0.8650
Epoch 117/3000
 - 0s - loss: 0.4775 - mean_squared_error: 0.1480 - acc: 0.8839 - val_loss: 0.5042 - val_mean_squared_error: 0.1604 - val_acc: 0.8650
Epoch 118/3000
 - 0s - loss: 0.4776 - mean_squared_error: 0.1481 - acc: 0.8741 - val_loss: 0.5035 - val_mean_squared_error: 0.1601 - val_acc: 0.8650
Epoch 119/3000
 - 0s - loss: 0.4770 - mean_squared_error: 0.1477 - acc: 0.8912 - val_loss: 0.5030 - val_me

Epoch 168/3000
 - 0s - loss: 0.4274 - mean_squared_error: 0.1266 - acc: 0.9071 - val_loss: 0.4624 - val_mean_squared_error: 0.1430 - val_acc: 0.8686
Epoch 169/3000
 - 0s - loss: 0.4281 - mean_squared_error: 0.1271 - acc: 0.9022 - val_loss: 0.4612 - val_mean_squared_error: 0.1425 - val_acc: 0.8686
Epoch 170/3000
 - 0s - loss: 0.4283 - mean_squared_error: 0.1274 - acc: 0.8875 - val_loss: 0.4600 - val_mean_squared_error: 0.1420 - val_acc: 0.8686
Epoch 171/3000
 - 0s - loss: 0.4271 - mean_squared_error: 0.1271 - acc: 0.8924 - val_loss: 0.4586 - val_mean_squared_error: 0.1415 - val_acc: 0.8686
Epoch 172/3000
 - 0s - loss: 0.4263 - mean_squared_error: 0.1268 - acc: 0.8888 - val_loss: 0.4575 - val_mean_squared_error: 0.1410 - val_acc: 0.8686
Epoch 173/3000
 - 0s - loss: 0.4233 - mean_squared_error: 0.1254 - acc: 0.8998 - val_loss: 0.4564 - val_mean_squared_error: 0.1406 - val_acc: 0.8686
Epoch 174/3000
 - 0s - loss: 0.4225 - mean_squared_error: 0.1253 - acc: 0.8973 - val_loss: 0.4555 - val_me

Epoch 223/3000
 - 0s - loss: 0.3871 - mean_squared_error: 0.1117 - acc: 0.9010 - val_loss: 0.4269 - val_mean_squared_error: 0.1291 - val_acc: 0.8686
Epoch 224/3000
 - 0s - loss: 0.3823 - mean_squared_error: 0.1094 - acc: 0.9059 - val_loss: 0.4265 - val_mean_squared_error: 0.1289 - val_acc: 0.8686
Epoch 225/3000
 - 0s - loss: 0.3841 - mean_squared_error: 0.1097 - acc: 0.9095 - val_loss: 0.4260 - val_mean_squared_error: 0.1288 - val_acc: 0.8686
Epoch 226/3000
 - 0s - loss: 0.3811 - mean_squared_error: 0.1087 - acc: 0.9144 - val_loss: 0.4254 - val_mean_squared_error: 0.1285 - val_acc: 0.8686
Epoch 227/3000
 - 0s - loss: 0.3807 - mean_squared_error: 0.1086 - acc: 0.9156 - val_loss: 0.4248 - val_mean_squared_error: 0.1283 - val_acc: 0.8686
Epoch 228/3000
 - 0s - loss: 0.3815 - mean_squared_error: 0.1090 - acc: 0.9095 - val_loss: 0.4240 - val_mean_squared_error: 0.1280 - val_acc: 0.8686
Epoch 229/3000
 - 0s - loss: 0.3758 - mean_squared_error: 0.1066 - acc: 0.9095 - val_loss: 0.4232 - val_me

...

Epoch 1048/3000
 - 0s - loss: 0.1392 - mean_squared_error: 0.0307 - acc: 0.9694 - val_loss: 0.3040 - val_mean_squared_error: 0.0931 - val_acc: 0.8613
Epoch 1049/3000
 - 0s - loss: 0.1395 - mean_squared_error: 0.0307 - acc: 0.9670 - val_loss: 0.3040 - val_mean_squared_error: 0.0931 - val_acc: 0.8613
Epoch 1050/3000
 - 0s - loss: 0.1435 - mean_squared_error: 0.0323 - acc: 0.9633 - val_loss: 0.3040 - val_mean_squared_error: 0.0931 - val_acc: 0.8613
Epoch 1051/3000
 - 0s - loss: 0.1396 - mean_squared_error: 0.0307 - acc: 0.9682 - val_loss: 0.3040 - val_mean_squared_error: 0.0931 - val_acc: 0.8613
Epoch 1052/3000
 - 0s - loss: 0.1367 - mean_squared_error: 0.0301 - acc: 0.9658 - val_loss: 0.3040 - val_mean_squared_error: 0.0931 - val_acc: 0.8613
Epoch 1053/3000
 - 0s - loss: 0.1351 - mean_squared_error: 0.0290 - acc: 0.9682 - val_loss: 0.3039 - val_mean_squared_error: 0.0931 - val_acc: 0.8613
Epoch 1054/3000
 - 0s - loss: 0.1408 - mean_squared_error: 0.0315 - acc: 0.9645 - val_loss: 0.3039 -

Epoch 1103/3000
 - 0s - loss: 0.1335 - mean_squared_error: 0.0287 - acc: 0.9743 - val_loss: 0.3035 - val_mean_squared_error: 0.0932 - val_acc: 0.8650
Epoch 1104/3000
 - 0s - loss: 0.1359 - mean_squared_error: 0.0303 - acc: 0.9633 - val_loss: 0.3035 - val_mean_squared_error: 0.0932 - val_acc: 0.8650
Epoch 1105/3000
 - 0s - loss: 0.1270 - mean_squared_error: 0.0271 - acc: 0.9682 - val_loss: 0.3035 - val_mean_squared_error: 0.0932 - val_acc: 0.8650
Epoch 1106/3000
 - 0s - loss: 0.1331 - mean_squared_error: 0.0290 - acc: 0.9694 - val_loss: 0.3035 - val_mean_squared_error: 0.0932 - val_acc: 0.8650
Epoch 1107/3000
 - 0s - loss: 0.1337 - mean_squared_error: 0.0293 - acc: 0.9719 - val_loss: 0.3035 - val_mean_squared_error: 0.0932 - val_acc: 0.8650
Epoch 1108/3000
 - 0s - loss: 0.1383 - mean_squared_error: 0.0304 - acc: 0.9731 - val_loss: 0.3035 - val_mean_squared_error: 0.0932 - val_acc: 0.8650
Epoch 1109/3000
 - 0s - loss: 0.1324 - mean_squared_error: 0.0290 - acc: 0.9694 - val_loss: 0.3035 -

Epoch 1158/3000
 - 0s - loss: 0.1299 - mean_squared_error: 0.0288 - acc: 0.9658 - val_loss: 0.3033 - val_mean_squared_error: 0.0934 - val_acc: 0.8650
Epoch 1159/3000
 - 0s - loss: 0.1300 - mean_squared_error: 0.0287 - acc: 0.9731 - val_loss: 0.3033 - val_mean_squared_error: 0.0934 - val_acc: 0.8650
Epoch 1160/3000
 - 0s - loss: 0.1300 - mean_squared_error: 0.0286 - acc: 0.9743 - val_loss: 0.3033 - val_mean_squared_error: 0.0934 - val_acc: 0.8650
Epoch 1161/3000
 - 0s - loss: 0.1274 - mean_squared_error: 0.0278 - acc: 0.9756 - val_loss: 0.3033 - val_mean_squared_error: 0.0934 - val_acc: 0.8650
Epoch 1162/3000
 - 0s - loss: 0.1296 - mean_squared_error: 0.0282 - acc: 0.9694 - val_loss: 0.3033 - val_mean_squared_error: 0.0934 - val_acc: 0.8650
Epoch 1163/3000
 - 0s - loss: 0.1291 - mean_squared_error: 0.0281 - acc: 0.9694 - val_loss: 0.3033 - val_mean_squared_error: 0.0934 - val_acc: 0.8650
Epoch 1164/3000
 - 0s - loss: 0.1276 - mean_squared_error: 0.0272 - acc: 0.9731 - val_loss: 0.3033 -

Epoch 1213/3000
 - 0s - loss: 0.1246 - mean_squared_error: 0.0274 - acc: 0.9694 - val_loss: 0.3033 - val_mean_squared_error: 0.0935 - val_acc: 0.8650
Epoch 1214/3000
 - 0s - loss: 0.1234 - mean_squared_error: 0.0268 - acc: 0.9731 - val_loss: 0.3033 - val_mean_squared_error: 0.0935 - val_acc: 0.8650
Epoch 1215/3000
 - 0s - loss: 0.1227 - mean_squared_error: 0.0268 - acc: 0.9694 - val_loss: 0.3034 - val_mean_squared_error: 0.0936 - val_acc: 0.8650
Epoch 1216/3000
 - 0s - loss: 0.1221 - mean_squared_error: 0.0265 - acc: 0.9731 - val_loss: 0.3034 - val_mean_squared_error: 0.0936 - val_acc: 0.8650
Epoch 1217/3000
 - 0s - loss: 0.1212 - mean_squared_error: 0.0257 - acc: 0.9768 - val_loss: 0.3034 - val_mean_squared_error: 0.0936 - val_acc: 0.8650
Epoch 1218/3000
 - 0s - loss: 0.1239 - mean_squared_error: 0.0265 - acc: 0.9707 - val_loss: 0.3034 - val_mean_squared_error: 0.0936 - val_acc: 0.8650
Epoch 1219/3000
 - 0s - loss: 0.1196 - mean_squared_error: 0.0256 - acc: 0.9731 - val_loss: 0.3034 -

(['loss', 'mean_squared_error', 'acc'],
 [0.3033647326222301, 0.09362178442687014, 0.8649635040847055])

In [21]:
nn_preds = model.predict_classes(x_test_norm)
nn_pos = model.predict_proba(x_test_norm)

In [22]:
accuracy(nn_preds), rmse(nn_pos), auroc(nn_pos), logloss(nn_pos)

(0.8613138686131386,
 0.30866156917979454,
 0.9448924731182796,
 0.3071082929254639)

In [23]:
(0.8461538461538461,
 0.3220160590119148,
 0.927314497995894,
 0.32741931771805405)

(0.8461538461538461,
 0.3220160590119148,
 0.927314497995894,
 0.32741931771805405)

In [24]:
train_df['gbc_preds'] = gbc.predict_proba(x_train).T[1]
test_df['gbc_preds'] = gbc.predict_proba(x_test).T[1]
train_df['rfc_preds'] = rfc.predict_proba(x_train).T[1]
test_df['rfc_preds'] = rfc.predict_proba(x_test).T[1]
train_df['etc_preds'] = etc.predict_proba(x_train).T[1]
test_df['etc_preds'] = etc.predict_proba(x_test).T[1]
train_df['nn_preds'] = model.predict_proba(x_train_norm)
test_df['nn_preds'] = model.predict_proba(x_test_norm)

In [25]:
train_df.head()

,d.gone.neg.l1,d.gone.neg.l2,d.neg.frac.l3,week,camp.length,base.poll,y2000,y2002,y2004,y2006,...,deminc,dem.contrib.l1,dem.contrib.l2,rep.contrib.l1,rep.contrib.l2,d.gone.neg,gbc_preds,rfc_preds,etc_preds,nn_preds
1,0,0,0.000000,-11,23,45.744681,0,1,0,0,...,1,38350.0,126536.00,132545.00,102559.36,0,0.000121,0.006,0.0,0.042282
2,0,0,0.000000,-10,23,45.744681,0,1,0,0,...,1,58035.0,38350.00,522327.00,132545.00,0,0.000146,0.000,0.0,0.041140
3,1,0,0.000000,-8,23,45.744681,0,1,0,0,...,1,167280.0,67068.67,177739.44,138533.00,1,0.999920,0.998,1.0,0.959737
4,1,1,0.000000,-7,23,45.744681,0,1,0,0,...,1,389947.0,167280.00,543465.00,177739.44,1,0.999957,1.000,1.0,0.970611
5,1,1,0.071429,-6,23,45.744681,0,1,0,0,...,1,202734.0,389947.00,2415314.50,543465.00,1,0.999956,1.000,1.0,0.980239


In [26]:
#train_df.to_csv(f'{PATH}train_preds.csv')
#test_df.to_csv(f'{PATH}test_preds.csv')